# 🌐 State Visualization

> **Section 2: Visualization (11% of Exam)** | Part 2 of 3

---

## 🔧 Core Tools: `Statevector` vs `StatevectorSampler`

Before diving into visualizations, understand the **two fundamentally different ways** to get quantum data:

### 📖 Definitions

| Tool | `Statevector` | `StatevectorSampler` |
|------|---------------|----------------------|
| **Import** | `from qiskit.quantum_info import Statevector` | `from qiskit.primitives import StatevectorSampler` |
| **Purpose** | Get **exact quantum state** amplitudes | Simulate **measurement sampling** |
| **Output** | Complex amplitude vector `[α, β, γ, δ...]` | Counts dictionary `{'00': 512, '11': 488}` |
| **Measurements** | ❌ Circuit must **NOT** have measurements | ✅ Circuit **MUST** have measurements |
| **Randomness** | Deterministic (same result every time) | Probabilistic (varies with shots) |

### 📊 Visualization Compatibility Matrix

| Visualization | `Statevector` | `StatevectorSampler` (counts) |
|--------------|:-------------:|:-----------------------------:|
| `plot_state_city()` | ✅ | ❌ |
| `plot_state_qsphere()` | ✅ | ❌ |
| `plot_bloch_multivector()` | ✅ | ❌ |
| `plot_state_hinton()` | ✅ | ❌ |
| `plot_state_paulivec()` | ✅ | ❌ |
| `plot_histogram()` | ❌ | ✅ |
| `plot_distribution()` | ❌ | ✅ |

---

## Setup

Import all required libraries:

In [ ]:
# Enable inline plotting
%matplotlib inline

# Import Qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import (
    plot_bloch_multivector,
    plot_state_city,
    plot_state_qsphere,
    plot_state_hinton,
    plot_state_paulivec,  # Added!
    plot_histogram,
    plot_distribution      # Added!
)

import matplotlib.pyplot as plt

print("✅ All imports successful!")
print("\n📊 Statevector → State visualizations (city, qsphere, bloch, hinton, paulivec)")
print("📈 StatevectorSampler → Histogram visualizations (histogram, distribution)")

---

## 📊 Part 1: `plot_histogram()` - MOST COMMON! (EXAM CRITICAL)

### Why Histograms Are Essential

`plot_histogram()` is the **#1 most used** visualization function because:
1. ✅ Visualizes real measurement results (what you get from quantum hardware)
2. ✅ Shows probability distribution from multiple shots
3. ✅ Easy to compare different circuits
4. ✅ Works for any number of qubits

### 📖 Understanding `counts`

**Definition**: `counts` is a Python dictionary mapping measurement outcomes to frequencies.

```python
counts = {'00': 512, '11': 488}  # Example for Bell state
#          │     │
#          │     └── Number of times this outcome occurred
#          └── Measurement outcome (binary string)
```

### Basic Usage

In [ ]:
# Create a Bell state circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

print("Bell State Circuit:")
print(qc.draw())

# Run with Sampler
sampler = StatevectorSampler()
job = sampler.run([qc], shots=1000)
result = job.result()

# Extract counts
counts = result[0].data.c.get_counts()
print(f"\nMeasurement counts: {counts}")

# Visualize
plot_histogram(counts)

### ⚠️ EXAM TRAP #1: Can't Plot Result Object Directly!

**This is a very common exam question!**

```python
# ❌ WRONG - Can't plot result directly
plot_histogram(result)

# ❌ WRONG - Need to extract counts first
plot_histogram(result[0])

# ✅ CORRECT - Extract counts, then plot
counts = result[0].data.c.get_counts()
plot_histogram(counts)
```

**Remember**: `plot_histogram()` takes a **dictionary** of counts, not a result object!

### ⚠️ EXAM TRAP #2: StatevectorSampler REQUIRES Measurements!

```python
# ❌ WRONG - No measurements in circuit
qc = QuantumCircuit(2)
qc.h(0)
sampler = StatevectorSampler()
job = sampler.run([qc])  # No counts to get!

# ✅ CORRECT - Add measurements
qc.measure_all()
job = sampler.run([qc], shots=1000)
counts = job.result()[0].data.c.get_counts()
```

### Histogram Customization Options

**Key parameters you should know for the exam:**

In [ ]:
# Customized histogram
plot_histogram(
    counts,
    title='Bell State Measurements',
    figsize=(10, 6),
    color='midnightblue',
    bar_labels=True,    # Show count values on bars
    sort='desc'         # Sort by count (desc/asc/value)
)

### Comparing Multiple Results

**Very useful for algorithm comparison!**

In [ ]:
# Create two different circuits
qc1 = QuantumCircuit(2, 2)
qc1.h([0, 1])  # Superposition
qc1.measure([0, 1], [0, 1])

qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.cx(0, 1)  # Entanglement
qc2.measure([0, 1], [0, 1])

# Run both
job1 = sampler.run([qc1], shots=1000)
job2 = sampler.run([qc2], shots=1000)

counts1 = job1.result()[0].data.c.get_counts()
counts2 = job2.result()[0].data.c.get_counts()

# Compare
plot_histogram(
    [counts1, counts2],
    legend=['Superposition', 'Entangled'],
    title='Comparison: Superposition vs Entanglement'
)

---

## 🌐 Part 2: Bloch Sphere Visualization (Single Qubits)

### The Bloch Sphere

The Bloch sphere represents **single-qubit states** geometrically:

```
         |0⟩ (North Pole)
          ↑
          |
     ┌────+────┐
    /     |     \
   /      |      \
  /       |       \
 |        |        |
  ────────+────────  ← |+⟩, |-⟩, |+i⟩, |-i⟩ (equator)
 |        |        |
  \       |       /
   \      |      /
    \     |     /
     └────+────┘
          |
          ↓
         |1⟩ (South Pole)

Key Points:
• |0⟩ at North Pole (top)      θ=0
• |1⟩ at South Pole (bottom)   θ=π
• |+⟩, |-⟩ on X-axis           θ=π/2, φ=0 or π
• |+i⟩, |-i⟩ on Y-axis         θ=π/2, φ=π/2 or 3π/2

Mathematical Form:
|ψ⟩ = cos(θ/2)|0⟩ + e^(iφ)sin(θ/2)|1⟩
```

### Using `plot_bloch_multivector()`

**a) WHEN TO USE**: When examining how each qubit behaves independently in a multi-qubit system

**b) WHERE IT HELPS**: Detecting entanglement (mixed states appear), teaching single-qubit rotations

**c) PURPOSE**: Visualizes each qubit's reduced state on its own Bloch sphere

In [ ]:
# # Create |+⟩ state
qc_plus = QuantumCircuit(1)
qc_plus.h(0)

# Get statevector
state = Statevector(qc_plus)

print("|+⟩ state on Bloch sphere:")
plot_bloch_multivector(state)

### Common States on Bloch Sphere

In [ ]:
# Visualize multiple common states
states_to_visualize = [
    ('|0⟩', QuantumCircuit(1)),
    ('|1⟩', QuantumCircuit(1)),
    ('|+⟩', QuantumCircuit(1)),
    ('|-⟩', QuantumCircuit(1)),
]

# Apply gates
states_to_visualize[1][1].x(0)  # |1⟩
states_to_visualize[2][1].h(0)  # |+⟩
states_to_visualize[3][1].x(0)
states_to_visualize[3][1].h(0)  # |-⟩

# Create subplots
fig, axes = plt.subplots(1, 4, figsize=(16, 4), subplot_kw={'projection': '3d'})

for idx, (title, qc) in enumerate(states_to_visualize):
    state = Statevector(qc)
    plot_bloch_multivector(state, title=title)
    
plt.tight_layout()
plt.show()

### ⚠️ EXAM TRAP #3: Bloch Sphere Shows Entanglement as "Mixed States"!

**Important**: For entangled states, individual Bloch spheres appear at the **center** (mixed state)!

```python
# ⚠️ WARNING: This shows EACH qubit separately
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)  # Entangled Bell state!

state = Statevector(qc)
plot_bloch_multivector(state)
# Shows 2 Bloch spheres with arrows at CENTER
# This indicates entanglement (qubits are correlated)!
```

**Interpretation**:
- Arrow at **pole** → Pure state (|0⟩, |1⟩, |+⟩, etc.)
- Arrow at **center** → Mixed state / part of entangled system
- Arrow on **equator** → Superposition state

**For entangled/multi-qubit states**: Use `plot_state_city()` or `plot_state_qsphere()` to see the full picture!

---

## 🏙️ Part 3: Multi-Qubit State Visualization

### `plot_state_city()` - 3D Bar Chart

**a) WHEN TO USE**: When you need to see the precise numerical values of state amplitudes

**b) WHERE IT HELPS**: Debugging algorithms, verifying state preparation, educational demos

**c) PURPOSE**: Displays complex amplitudes as 3D bars - real & imaginary parts separately

**Shows**: Real and imaginary parts as 3D bars

In [ ]:
# Create Bell state
qc_bell = QuantumCircuit(2)
qc_bell.h(0)
qc_bell.cx(0, 1)

state = Statevector(qc_bell)

print("Bell State: (|00⟩ + |11⟩)/√2")
print(f"\nState vector: {state}")
print("\n3D City Plot:")
plot_state_city(state, title='Bell State Amplitudes')


**Interpretation of plot_state_city()**:
- **X-axis**: Basis states (|00⟩, |01⟩, |10⟩, |11⟩)
- **Y-axis**: Real and Imaginary parts
- **Bar height**: Amplitude magnitude
- **Bar color**: Phase (color map)

For Bell state:
- Large bars at |00⟩ and |11⟩ (Real part = 1/√2)
- No bars at |01⟩ and |10⟩ (amplitude = 0)
- No imaginary components

### `plot_state_qsphere()` - Q-Sphere

**a) WHEN TO USE**: When analyzing multi-qubit states where phase relationships matter

**b) WHERE IT HELPS**: Understanding superposition patterns, detecting phase errors in algorithms

**c) PURPOSE**: Shows probability (marker size) and phase (color) on a spherical surface

**Shows**:
- Marker size ∝ probability
- Marker color = phase
- Position = basis state

In [ ]:
# Q-sphere visualization
plot_state_qsphere(state, title='Bell State Q-Sphere')

### `plot_state_hinton()` - Density Matrix

**a) WHEN TO USE**: When analyzing density matrices, entanglement, or mixed states

**b) WHERE IT HELPS**: Identifying off-diagonal coherences, spotting decoherence effects

**c) PURPOSE**: Displays density matrix elements as squares - size shows magnitude, color shows sign

**Shows**:
- Square size = element magnitude
- White = positive, Black = negative
- Useful for detecting entanglement (off-diagonal elements)

In [ ]:
# Hinton plot
plot_state_hinton(state, title='Bell State Density Matrix')

### `plot_state_paulivec()` - Pauli Vector Decomposition

**a) WHEN TO USE**: When working with Hamiltonians or Pauli-based decompositions

**b) WHERE IT HELPS**: VQE algorithms, quantum chemistry, error analysis via Pauli channels

**c) PURPOSE**: Shows expectation values of all Pauli operator combinations (II, IX, IY, IZ, XI, XX, ...)

In [ ]:
# Pauli vector plot
plot_state_paulivec(state, title='Bell State Pauli Decomposition')

---

## 📊 Part 4: Which Visualization to Use?

### Decision Tree

```
┌─ Have measurement results (counts)? ─────────┐
│  → plot_histogram(counts)                    │
│     ✅ MOST COMMON CASE!                     │
└──────────────────────────────────────────────┘

┌─ Single qubit state? ────────────────────────┐
│  → plot_bloch_multivector(state)             │
│     (Shows geometric representation)         │
└──────────────────────────────────────────────┘

┌─ Multi-qubit state (need amplitudes)? ──────┐
│  → plot_state_city(state)                    │
│     (Shows real/imaginary parts)             │
└──────────────────────────────────────────────┘

┌─ Multi-qubit state (need probabilities)? ───┐
│  → plot_state_qsphere(state)                 │
│     (Shows probabilities and phases)         │
└──────────────────────────────────────────────┘

┌─ Need to see entanglement/correlations? ────┐
│  → plot_state_hinton(state)                  │
│     (Shows density matrix)                   │
└──────────────────────────────────────────────┘

┌─ Working with Hamiltonians/VQE? ────────────┐
│  → plot_state_paulivec(state)                │
│     (Shows Pauli operator expectations)      │
└──────────────────────────────────────────────┘
```

### Visual Summary of All State Visualizations

```
┌─────────────────────────────────────────────────────────────────┐
│ plot_state_city()           │ plot_state_qsphere()             │
│ ┌────────────┐              │         ○                        │
│ │ ▓▓   ▓▓    │ Real         │       ●   ●                      │
│ │            │              │     ○       ○                    │
│ │            │ Imag         │       ●   ●   (sphere with       │
│ └────────────┘              │         ○      markers)          │
│ 3D bars for amplitudes      │ Probability + phase on sphere    │
├─────────────────────────────┼──────────────────────────────────┤
│ plot_bloch_multivector()    │ plot_state_hinton()              │
│   ↗                ↗        │ ┌──┬──┬──┬──┐                    │
│  ●                ●         │ │▓▓│  │  │▓▓│                    │
│ ↙                ↙          │ │  │  │  │  │                    │
│ Qubit 0        Qubit 1      │ │  │  │  │  │                    │
│ Individual Bloch spheres    │ │▓▓│  │  │▓▓│                    │
│                             │ └──┴──┴──┴──┘                    │
│                             │ Density matrix squares           │
├─────────────────────────────┴──────────────────────────────────┤
│ plot_state_paulivec()                                          │
│ ┌────┬────┬────┬────┬────┬────┬────┬────┐                      │
│ │ II │ IX │ IY │ IZ │ XI │ XX │ XY │... │                      │
│ │ ▓▓ │    │    │    │    │ ▓▓ │    │    │                      │
│ └────┴────┴────┴────┴────┴────┴────┴────┘                      │
│ Pauli operator expectation values                              │
└─────────────────────────────────────────────────────────────────┘
```

### Function Comparison Table

| Function | Input | Best For | When/Where/Purpose |
|----------|-------|----------|-------------------|
| `plot_histogram()` | Counts dict | Measurement results | Real experiments, validating results |
| `plot_bloch_multivector()` | Statevector | Geometric view | Single qubits, detecting entanglement |
| `plot_state_city()` | Statevector | Amplitudes (real/imag) | Debugging, exact amplitude values |
| `plot_state_qsphere()` | Statevector | Probabilities + phase | Phase relationships, superposition |
| `plot_state_hinton()` | Statevector | Entanglement/correlations | Density matrices, coherences |
| `plot_state_paulivec()` | Statevector | Pauli decomposition | VQE, quantum chemistry |

---

## 🎯 Part 5: Complete Example - Algorithm Verification

### Verifying a Quantum Algorithm

Let's verify a 3-qubit GHZ state using multiple visualizations:

In [ ]:
# Create GHZ state: (|000⟩ + |111⟩)/√2
qc_ghz = QuantumCircuit(3, 3)
qc_ghz.h(0)
qc_ghz.cx(0, 1)
qc_ghz.cx(1, 2)
qc_ghz.barrier()
qc_ghz.measure(range(3), range(3))

print("GHZ State Circuit:")
print(qc_ghz.draw())

In [ ]:
# Get statevector (before measurement)
qc_ghz_state = QuantumCircuit(3)
qc_ghz_state.h(0)
qc_ghz_state.cx(0, 1)
qc_ghz_state.cx(1, 2)

state_ghz = Statevector(qc_ghz_state)
print("GHZ State vector:")
print(state_ghz)
print("\nExpected: (|000⟩ + |111⟩)/√2")
print("         = 0.707|000⟩ + 0.707|111⟩")

In [ ]:
# Visualization 1: Bloch spheres (each qubit)
print("Individual qubit view (Bloch spheres):")
plot_bloch_multivector(state_ghz)

In [ ]:
# Visualization 2: State city (amplitudes)
print("Amplitude view (3D bars):")
plot_state_city(state_ghz, title='GHZ State Amplitudes')

In [ ]:
# Visualization 3: Q-sphere (probabilities)
print("Probability view (Q-sphere):")
plot_state_qsphere(state_ghz)
plt.title('GHZ State Q-Sphere')

In [ ]:
# Visualization 4: Measurement results (histogram)
job_ghz = sampler.run([qc_ghz], shots=1000)
counts_ghz = job_ghz.result()[0].data.meas.get_counts()

print("Measurement results:")
plot_histogram(counts_ghz, title='GHZ State Measurements')

print(f"\nCounts: {counts_ghz}")
print("Expected: ~500 counts of |000⟩ and ~500 counts of |111⟩")
print("(Perfect 3-qubit correlation!)")

---

## 📝 Practice Questions (EXAM STYLE)

### Question 1: Most Common Visualization

**After running a circuit on a quantum computer and getting results, which visualization function should you use?**

A) `plot_bloch_multivector()`  
B) `plot_histogram()`  
C) `plot_state_city()`  
D) `plot_state_qsphere()`

<details>
<summary>Click to reveal answer</summary>

**Answer: B) `plot_histogram()`**

**Explanation**:
- Quantum computer measurements give you **counts** (classical bit strings)
- `plot_histogram()` visualizes measurement counts
- This is the MOST COMMON visualization in practice
- Other functions need Statevector (simulation only)

**Usage**:
```python
counts = result[0].data.meas.get_counts()
plot_histogram(counts)
```
</details>

---

### Question 2: Statevector vs StatevectorSampler

**Which statement is TRUE about `Statevector` and `StatevectorSampler`?**

A) Both require measurements in the circuit  
B) `Statevector` returns counts, `StatevectorSampler` returns amplitudes  
C) `Statevector` gives exact amplitudes, `StatevectorSampler` simulates measurements  
D) They can be used interchangeably

<details>
<summary>Click to reveal answer</summary>

**Answer: C) `Statevector` gives exact amplitudes, `StatevectorSampler` simulates measurements**

**Key differences**:
| | `Statevector` | `StatevectorSampler` |
|---|---|---|
| Output | Complex amplitudes | Counts dictionary |
| Measurements | Must NOT have | MUST have |
| Randomness | Deterministic | Probabilistic |

```python
# Statevector - no measurements!
state = Statevector(qc)  # Returns [0.707, 0, 0, 0.707]

# StatevectorSampler - needs measurements!
qc.measure_all()
counts = sampler.run([qc]).result()[0].data.meas.get_counts()
# Returns {'00': 512, '11': 488}
```
</details>

---

### Question 3: Wrong Visualization Error

**What happens when you run `plot_state_city(counts)` where `counts = {'00': 500, '11': 500}`?**

A) It works correctly  
B) TypeError - expects Statevector not dict  
C) It shows a histogram  
D) It shows empty bars

<details>
<summary>Click to reveal answer</summary>

**Answer: B) TypeError - expects Statevector not dict**

**Explanation**:
- `plot_state_city()` expects a `Statevector` object
- `counts` is a dictionary (classical measurement results)
- You cannot use state visualizations with measurement counts!

**Correct usage**:
```python
# For counts → use histogram
plot_histogram(counts)

# For state → use state visualizations
state = Statevector(qc)  # No measurements!
plot_state_city(state)
```
</details>

---

### Question 4: Bloch Sphere for Entangled State

**When you visualize an entangled Bell state using `plot_bloch_multivector()`, what do you see?**

A) Two spheres with arrows pointing north  
B) Two spheres with arrows pointing in random directions  
C) Two spheres with arrows at the center (origin)  
D) A single sphere showing the combined state

<details>
<summary>Click to reveal answer</summary>

**Answer: C) Two spheres with arrows at the center (origin)**

**Explanation**:
- Entangled qubits cannot be described independently
- When you trace out one qubit, the other appears "mixed"
- Mixed states show arrows at the **center** of the Bloch sphere
- This is actually how you can **detect entanglement** visually!

```python
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)  # Entangled!
state = Statevector(qc)
plot_bloch_multivector(state)  # Arrows at center!
```
</details>

## ✅ Key Takeaways - Master These for the Exam!

### Core Concepts

1. **Two Ways to Get Data**
   - `Statevector` → Exact quantum state (no measurements)
   - `StatevectorSampler` → Simulated measurements (needs measurements)

2. **Visualization Compatibility**
   - State visualizations (`city`, `qsphere`, `bloch`, `hinton`, `paulivec`) → Need `Statevector`
   - Result visualizations (`histogram`, `distribution`) → Need `counts` dict

3. **All 5 State Visualizations**
   | Function | When to Use |
   |----------|-------------|
   | `plot_bloch_multivector()` | Single qubits, detect entanglement |
   | `plot_state_city()` | See exact amplitudes |
   | `plot_state_qsphere()` | See probabilities + phase |
   | `plot_state_hinton()` | See density matrix |
   | `plot_state_paulivec()` | Pauli decomposition |

### ⚠️ Critical Exam Traps

| Trap | Wrong | Correct |
|------|-------|---------|
| Plot result directly | `plot_histogram(result)` | `plot_histogram(counts)` |
| State viz with counts | `plot_state_city(counts)` | `plot_state_city(Statevector(qc))` |
| Histogram with state | `plot_histogram(state)` | `plot_histogram(counts)` |
| Statevector with measurements | `Statevector(qc_with_meas)` | `Statevector(qc_no_meas)` |
| Sampler without measurements | `sampler.run([qc_no_meas])` | `sampler.run([qc_with_meas])` |

### 🧠 Memory Aids

**"State = Statevector, Sample = Sampler"**
- Statevector = The quantum state itself (what it IS)
- StatevectorSampler = Samples from the state (what we MEASURE)

**"City Qsphere Bloch Hinton Pauli → State"** (All need Statevector)
**"Histogram Distribution → Counts"** (All need counts dict)

### Quick Decision Flowchart

```
Do you have measurements? ─┬─ YES → counts dict → plot_histogram()
                           │
                           └─ NO → Statevector → plot_state_city()
                                                 plot_state_qsphere()
                                                 plot_bloch_multivector()
                                                 plot_state_hinton()
                                                 plot_state_paulivec()
```

---

## 🔗 What's Next?

**Visualization Examples** (`visualization_examples.ipynb`) - Comprehensive practice with:
- Superposition vs Entanglement visualization
- All visualization functions side-by-side
- Exam-style practice scenarios

**Key difference**: State visualization shows the **quantum state** BEFORE measurement. Histogram shows **what we observe** AFTER measurement!